# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# Libraries
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [2]:
# your code here


admissions = pd.read_csv(r'C:\Users\chris\lab-advanced-pandas\your-code\data\Admission_Predict.csv')


admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Remove trailing spaces at the end of the column names if there are any.


In [4]:
# your code here

#remove the trailing spaces using the rstrip method on the dataframe columns
admissions.columns = admissions.columns.str.rstrip()


Let's evaluate the dataset by looking at the `head` function.

In [5]:
# your code here
admissions.head()


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [6]:
# your code here

#use the isnull() and any() method which returns a True if there are any null values (and false if not)
admissions.isnull().values.any()

False

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [7]:
# your code here


admissions.set_index('Serial No.')

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [8]:
# your code here

#There are 385 rows so if they have unique values then dropping duplicates should retreieve the same amount of rows

admissions['GRE Score'].drop_duplicates() 

#it does not though? 

0      337
1      316
2      322
3      314
4      330
5      321
6      308
7      302
8      323
9      327
10     328
11     307
12     311
14     317
15     319
16     318
17     303
18     325
20     334
21     336
23     298
24     295
25     310
26     300
28     338
29     340
30     331
31     320
32     299
34     304
38     313
39     332
40     326
42     329
43     339
47     312
49     324
56     309
59     315
73     301
74     296
75     294
91     306
105    305
112    290
115    335
124    333
152    297
161    293
Name: GRE Score, dtype: int64

In [10]:

# Trying a different solution using duplicated() method
# This method returns a boolean and we need to sum it up to figure out how many duplicated do we have.
# This time it shows that we have zero duplicates, hence these two columns uniquely identify the data I think

admissions.duplicated(subset=["GRE Score", "CGPA"]).sum()

0

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [15]:
# your code here


admissions_df = admissions.set_index(['GRE Score', 'CGPA'])
admissions_df.reset_index(['GRE Score', 'CGPA'], inplace=True)
admissions_df.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
1,316,8.00,2,104,3,3.0,3.5,1,0.72
2,322,8.67,3,110,3,3.5,2.5,1,0.80
3,314,8.21,4,103,2,2.0,3.0,0,0.65
4,330,9.34,5,115,5,4.5,3.0,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [16]:
reseach_and_gpa = admissions_df.loc[(admissions_df['CGPA'] > 9) & (admissions_df['Research'] == 1), ['CGPA','Research']]

reseach_and_gpa.head()

,CGPA,Research
0,9.65,1
4,9.34,1
10,9.10,1
19,9.50,1
20,9.70,1


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [56]:
# your code here

admissions_df.loc[(admissions_df['CGPA'] > 9) & (admissions_df['SOP'] < 3.5), ['CGPA','SOP', 'Chance of Admit']]
admissions_df.head()


,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
1,316,8.00,2,104,3,3.0,3.5,1,0.72
2,322,8.67,3,110,3,3.5,2.5,1,0.80
3,314,8.21,4,103,2,2.0,3.0,0,0.65
4,330,9.34,5,115,5,4.5,3.0,1,0.90


In [57]:
mean = cgpa_and_sop['Chance of Admit'].mean()
print(mean)

0.8019999999999999


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [58]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """

    return (col - np.mean(col)) / np.std(col)
    
    # your code here

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [59]:
# your code here

## Using assign() method and lambdas to add new standardized columns

admissions_std = admissions_df.assign(CGPA_std=lambda x: standardize(x["CGPA"]),
                               
                               GRE_std=lambda x: standardize(x["GRE Score"]),
                               
                               LOR_std=lambda x: standardize(x["LOR"])
                              )


admissions_std.head() 

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338


We will generate the decision choice at random using the code below. Please run the cell.

In [60]:
# Libraries
from random import choices

In [61]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [62]:
# your code here

admissions_std["deciding_column"] = admissions_std.lookup(admissions_std.index, decision_choice)

admissions_std.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.750174
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.992501
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,0.456297
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,1.149292


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [63]:
# your code here

# with the np.where() method we test the condition if "deciding_colum" is more than 0.8 then new value is 1, otherwise 0.

admissions_std["decision"] = np.where(admissions_std["deciding_column"] > 0.8, 1, 0)

admissions_std.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.750174,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,0.456297,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,1.149292,1


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [64]:
# your code here


## 300 are not admited, 85 are admited

admissions_std["decision"].value_counts()

0    293
1     92
Name: decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [65]:
# your code here

# Here, we chained Pandas Series string methods

admissions_std.columns = (admissions_std.columns
                                .str.replace(" ", "_")
                                .str.replace(".", "")
                                .str.lower())

In [66]:
admissions_std.columns

Index(['gre_score', 'cgpa', 'serial_no', 'toefl_score', 'university_rating',
       'sop', 'lor', 'research', 'chance_of_admit', 'cgpa_std', 'gre_std',
       'lor_std', 'deciding_column', 'decision'],
      dtype='object')

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [67]:
# your code here


# Using np.where() method to check the condition if university rating is more or equal to 4 and
# if this condition is true then we add 10 to GRE Score column. Otherwise GRE Score column is untouched.
# After that we use Pandas cut method() to cut new column 4 equal withd bin.

admissions_std["adjusted_gre"] = pd.cut(np.where(admissions_std["university_rating"] >= 4,
                                             admissions_std["gre_score"] + 10,
                                             admissions_std["gre_score"]),
                                    bins=4)

In [68]:
#check to see it's working

admissions_std.head()

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.750174,1,"(335.0, 350.0]"
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,0,"(305.0, 320.0]"
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,0.456297,0,"(320.0, 335.0]"
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.643433,0,"(305.0, 320.0]"
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,1.149292,1,"(335.0, 350.0]"


In [69]:
#assign it to a variable

adjusted_gre = admissions_std["adjusted_gre"]
print(adjusted_gre)

0      (335.0, 350.0]
1      (305.0, 320.0]
2      (320.0, 335.0]
3      (305.0, 320.0]
4      (335.0, 350.0]
            ...      
380    (320.0, 335.0]
381    (320.0, 335.0]
382    (335.0, 350.0]
383    (305.0, 320.0]
384    (335.0, 350.0]
Name: adjusted_gre, Length: 385, dtype: category
Categories (4, interval[float64]): [(289.94, 305.0] < (305.0, 320.0] < (320.0, 335.0] < (335.0, 350.0]]
